In [7]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio, typing
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict, List
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

message = "Latest AI Agent framework in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)
    
display(Markdown(result.final_output))

As of July 2025, several advanced AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**LangChain** has become a leading platform for building language-powered agents, enabling developers to create complex workflows by chaining prompts, memory, and tools. Its modular components and integrated memory support make it ideal for applications like conversational AI and automated research assistants. ([lekha-bhan88.medium.com](https://lekha-bhan88.medium.com/top-5-agentic-ai-frameworks-to-watch-in-2025-9d51b2b652c0?utm_source=openai))

**LangGraph**, an extension of LangChain, focuses on stateful, graph-based orchestration of multi-agent systems. It allows for explicit control over information flow, supporting complex workflows with branching logic and human intervention points, suitable for applications requiring detailed state management and decision-making processes. ([radarmagazine.com](https://www.radarmagazine.com/top-5-ai-agent-frameworks-it-executives-should-be-watching-in-2025/?utm_source=openai))

**CrewAI** adopts a role-based approach to agent collaboration, facilitating the creation of specialized agents that work together on complex projects. Its dynamic task planning and real-time performance monitoring make it well-suited for scenarios like project management simulations and healthcare coordination. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**AutoGen**, developed by Microsoft, specializes in orchestrating multiple AI agents to form autonomous, event-driven systems capable of handling complex, multi-agent tasks seamlessly. It is ideal for enterprise-level applications needing event-driven workflows, such as customer service automation. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**Eliza** is an open-source, Web3-friendly AI agent operating system that seamlessly integrates with blockchain applications. It allows for the deployment of Web3 applications within AI agents, enabling functionalities like reading and writing blockchain data and interacting with smart contracts. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

These frameworks represent the forefront of AI agent development, each catering to different needs and applications in the rapidly evolving field of artificial intelligence. 

In [9]:
HOW_MANY_SEARCHES = 3
INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: List[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    output_type=WebSearchPlan,
    model="gpt-4o-mini",
)

In [10]:
message = "latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find current information on AI agent frameworks being developed or released in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To gain insights into features and advancements in AI agent technology for 2025.', query='AI agent technology advancements 2025'), WebSearchItem(reason='To explore comparisons or reviews of various AI agent frameworks available in 2025.', query='comparison of AI agent frameworks 2025')]


In [11]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gtimokhinas@proton.me") # Change this to your verified email
    to_email = To("gtimokhinas@proton.me") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [12]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115b81b20>, strict_json_schema=True, is_enabled=True)

In [13]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [14]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

3 functions to plan and execute the search, using planner_agent and search_agent

In [15]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

Next 2 functions to write a report and email it

In [16]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

And run everything

In [17]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


check the trace : https://platform.openai.com/traces